In [1]:
import pandas as pd
import numpy as np

In [2]:
guests = pd.read_excel("data/guests.xlsx", index_col=0)
guests.drop("guest", axis= 1, inplace = True)
guests = 1 - guests 
vec_discount = guests["discount"].values #converts the discount column to a vector

In [3]:
hotels = pd.read_excel("data/hotels.xlsx", index_col=0)
vec_prices = hotels["price"].values

In [4]:
#Viene creata un Dataframe, ma adesso gli hotel sono le rows e i guest le columns.
preferences = pd.read_excel("data/preferences.xlsx", index_col=0)
preferences["guest"] = pd.to_numeric(preferences["guest"].str.replace('guest_','')) - 1
preferences["hotel"] = pd.to_numeric(preferences["hotel"].str.replace('hotel_','')) - 1
pref_pivoted_o = pd.pivot_table(preferences, index ='guest', columns ='hotel', values= "priority" , aggfunc="min") #it pivots the original dataframe to get to a matrix-like format
rank_matrix_o = pref_pivoted_o.rank(axis = 1, method="dense").values #reorders preferences
#pref_pivoted_o =  pd.DataFrame(rank_matrix_o)

#pref_pivoted = pd.pivot_table(preferences, index ='hotel', columns ='guest', values= "priority" , aggfunc="min") #it pivots the original dataframe to get to a matrix-like format
#rank_matrix = pref_pivoted.rank(axis = 0, method="dense").values #reorders preferences
#pref_pivoted =  pd.DataFrame(rank_matrix)

In [ ]:
h_rooms

In [8]:
#dup = h_rooms ##### .save
def room_model(hotels,rank_matrix):

    pref_pivoted=pd.DataFrame(rank_matrix).T
    pref_pivoted["price"] = hotels["price"]
    pref_pivoted["rooms"] =  hotels["rooms"] #add a column with number of rooms per hotel
    h_rooms = pref_pivoted.sort_values(by=['rooms','price'], ascending=[False,True]) #sorted by number of rooms
    h_rooms = h_rooms.drop(columns=["rooms","price"]) # drop column with number of rooms
    dup = h_rooms



    choice_matrix = np.zeros(rank_matrix.shape) 
    v= hotels["rooms"].values

    for i,r in dup.iterrows():
        x=r.sort_values()
        x=x.dropna()
        c=x[:v[i]]
        for idx in c.index:
            choice_matrix[idx][i] = 1
            #dup.drop([idx], axis=1) ??????
            dup.iloc[:][idx]= np.nan
        
        
        
        
    return  choice_matrix








In [10]:
room_model(hotels,rank_matrix_o).sum()

4000.0

In [ ]:
choice_matrix.sum(axis=1).sum()

In [ ]:
cm = pd.DataFrame(choice_matrix)
cm.iloc[:][48].sum()

In [ ]:
h_rooms #?????????????

In [ ]:
choice_matrix[:,48]==choice_matrix[48,:]

#Viene creato un dataframe dove ogni rows corrispondente a un hotel viene ripetuta per il numero di camere.
#l'indice delle rows viene replicato in modo da mantenere quello originale degli hotel.

data = pd.DataFrame(np.repeat(h_rooms.values, h_sorted["rooms"], axis=0)) 
index = pd.DataFrame(np.repeat(h_rooms.index.values, h_sorted["rooms"], axis=0))
data = np.nan_to_num(data)
data = pd.DataFrame(data, index=index[0])
data

 L'idea alla base di questo dataframe è che per ogni riga
#si prende il numero più basso n (che corrisponde alla maggior preferenza) 
#e in caso di n>1 si sceglie quello con l'indice di colonna minore (che corrisponde all'ordine di prenotazione).
#Il valore scelto viene trasformato in 1 mentre il resto della colonna in 0.
 Usando gli indici del dataframe (400 per le rows, 4000 per le columns) il valore scelto viene riportato nella choice_matrix (4000X400).


In [ ]:
#Hic Sunt Leones

obiettivo: choice_matrix 4000X400
hotels abbiamo: indice, camere, prezzo


In [ ]:
#This function creates a utility matrix from the ranked preferences matrix
def to_utility_matrix(rank_matrix):
  user_utility = []
  for row in rank_matrix:
    utility_values = np.linspace(start=1, stop= 0.1, num=len(set(row[~np.isnan(row)]))) #utility goes from 1 to 0.1 split evenly between preferences
    user_utility.append([utility_values[int(i)-1] if i == i else i for i in row ]) #i == i to check if i is not nan 
  return np.nan_to_num(np.array(user_utility), nan = 0)

In [ ]:
utility_matrix = to_utility_matrix(rank_matrix)

In [ ]:
# def print_output(res, guest_count, room_count, hotel_count, t):
#   res_avg = np.round_(np.array(res).mean(axis=0), 3)
#   print(f"With {t} iterations, on average we have: \n{res_avg[0]}/{guest_count} guests placed \n{res_avg[1]}/{room_count} rooms occupied \n{res_avg[2]}/{hotel_count} hotels occupied \n{res_avg[3]} total revenue \n{res_avg[4]} total satisfaction")

In [7]:
def results(choice_matrix, vec_prices=vec_prices, vec_discount=vec_discount, utility_matrix=utility_matrix):
  guests_placed = choice_matrix.sum(axis=1).sum()
  rooms_occupied = choice_matrix.sum(axis=0).sum()
  hotels_occupied = np.where(choice_matrix.sum(axis=0) > 0, 1, 0).sum()
  prices_matrix = vec_prices * choice_matrix #all 1s in the matrix will be replaced by that hotel's price
  discount_matrix = np.multiply(prices_matrix, vec_discount[:, np.newaxis]) #the room price is then discounted for each user
  revenue = discount_matrix.sum()
  utility = np.multiply(choice_matrix, utility_matrix).sum()
  return([guests_placed, rooms_occupied, hotels_occupied, revenue, utility])

NameError: name 'utility_matrix' is not defined

In [ ]:
def random_model(hotels, vec_prices, vec_discount, rank_matrix, t=1000):
  guest_count, room_count, hotel_count = rank_matrix.shape[0] , hotels["rooms"].sum(), rank_matrix.shape[1]
  least = min([guest_count, room_count])
  res = []
  vec_rooms = np.repeat(hotels["price"], hotels["rooms"]).index.values #Denormalized the dataframe by adding rows. This vec contains the hotel number as many times as its rooms 
  for _ in range(t):
    choice_matrix = np.zeros(rank_matrix.shape) #creates filled with zeros 4000 x 400
    random_rooms = np.random.choice(vec_rooms, least, replace=False ) #selects randomly 4000 rooms from all possible rooms
    choice_matrix[tuple(range(least)),tuple(random_rooms)] = 1 #assign 1 to all occupied rooms given their indices
    res.append(results(choice_matrix, vec_prices, vec_discount))
  return list(np.array(res).mean(axis=0))

In [ ]:
def preference_model(hotels, vec_prices, rank_matrix):
  rank_matrix_filled, choice_matrix  = np.nan_to_num(rank_matrix), np.zeros(rank_matrix.shape) 
  rooms = hotels["rooms"].values
  for idx, row in enumerate(rank_matrix_filled):
    c = np.multiply(row, np.where(rooms > 0, 1, 0) ) #sets preferences values to 0 where there is no room availability
    try:
      user_choice = np.where(c == np.min(c[np.nonzero(c)]), 1, 0) #The minimum nonzero is converted to 1 and the rest to 0
      choice_matrix[(idx),] = user_choice
      rooms = rooms - user_choice #subtract 1 from the room availability where it's now occupied 
    except ValueError: #pass if array are all zeros
     pass
  return  results(choice_matrix)

In [ ]:
random_model(hotels, vec_prices, vec_discount, rank_matrix, t=1000)

In [ ]:
preference_model(hotels, vec_prices, rank_matrix)